In [20]:
import numpy as np
import heapq
import random
THINK = 4
REQ = 5
IDLE = 6
BUSY = 7
ARR = 1
DEP = 2
SWCH = 3
np.random.seed(42)

In [21]:
class Queue:
    def __init__(self):
        self.len = 1000
        self.que = np.empty(self.len, Request)
        self.front = -1
        self.rear = -1
        
    def display(self):
        if(self.front == -1):
            print("Underflow")
            return
        else:
            i = self.front
            while(i != self.rear):
                print("user:",self.que[i].user)
                i = (i+1)%self.len
            print("user:",self.que[i].user)
            return
        
    def increase_waittime(self, clock, tstamp):
        i = self.front
        while(i != self.rear):
            self.que[i].add_to_wait(clock, tstamp)
            i = (i+1)%self.len
        self.que[i].add_to_wait(clock, tstamp)
    
    def isFull(self):
        return True if((self.rear+1)%self.len == self.front) else False 
                    
    def isEmpty(self):
        return True if(self.front == -1) else False 
    
    def enqueue(self,req):
        if((self.rear+1)%self.len == self.front):
            print("Overflow")
            return False
        if(self.rear == -1):
            self.front += 1
            self.rear += 1
        else:
            self.rear = (self.rear+1)%self.len
        self.que[self.rear] = req
        return True
    
    def dequeue(self):
        if(self.front == -1):
            print("Underflow")
            return False
        req = self.que[self.front]
        if(self.rear == self.front):
            self.front = -1
            self.rear = -1
        else:
            self.front = (self.front+1)%self.len
        return req

In [22]:
class Request:
    def __init__ (self,req_id, user_id, t_req, tp_timeout, tp_service):
#         print("Req init():",user_id,t_req,tp_service)
        self.id = req_id
        self.user = user_id
        self.t_req = t_req
        self.tp_timeout = tp_timeout
        self.tp_service = tp_service
        self.tp_wait = 0
        self.core_num = -1
        self.start_wait_t = t_req
        
    def add_to_wait(self,clock):
        self.tp_wait += clock - self.start_wait_t

    def assign_core(self, core_num):
#         print("core assigned:",core_num)
        self.core_num = core_num

    def dec_time(self, sw_time):
#         print("Decreasing time...","from",self.tp_service)
        self.tp_service = self.tp_service - sw_time
#         print("...to",self.tp_service)

In [50]:
class Simulation:
    def __init__(self, num_users):
        print("Simulation intialising...")
        self.clock = 0.0
        self.event_pq = []
        heapq.heapify(self.event_pq)
        
        self.que = Queue()
        self.s = Server(2)
        
        self.no_of_users = num_users
        self.user = []
        self.req_num = 0
        for i in range(self.no_of_users):
            self.user.append(Users(i, self))  
#         self.n_arrival, self.n_arr_u = self.generate_next_arrival()
        self.n_e_type = ARR
        self.n_depart = float('inf')
        
    
    def generate_next_arrival(self):
#         print("generate_next_arrival()...")
        t = self.user[0].t_req 
        u = 0
        for i in range(self.no_of_users):
            if (self.user[i].t_req <  t):
                t = self.user[i].t_req
                u = i
        return t,u
    
    def arrival_handler(self,event):
#         print("Handling arrival...:","user",event.req.user)
        # ask user to send request and put it in queue
        if not(self.user[event.req.user].put_in_queue(self)):
            return False
        # ask server to serve if IDLE
        self.s.serve(self, event)
#         self.n_arrival, self.n_arr_u = self.generate_next_arrival()
        
    def depart_handler(self,event):
#         print("Handling Departure...",event.e_type,event.req.user)
        self.s.serve(self, event)
#         self.n_arrival, self.n_arr_u = self.generate_next_arrival()

    def switch_handler(self, event):
#         print("Handling Switch...",event.e_type,event.req.user)
        self.s.serve(self, event)
        
    def advance_time(self, count):
        counter=0
        while(len(self.event_pq) != 0):
            event = heapq.heappop(self.event_pq)
            print("Event popped:",event.e_type,"of user",event.req.user)
            self.clock = event.timestamp
            if event.e_type == ARR :
                counter += 1
                self.arrival_handler(event)
            elif event.e_type == DEP :
                self.depart_handler(event)
            elif event.e_type == SWCH:
                self.switch_handler(event)
#             self.display()
#             return #for debugging, else loop till counter value
            if(counter == count):
                break
                
    def display(self):
        print("===================================================")
        print("Clock :",self.clock)
        for i in range(self.no_of_users):
            self.user[i].display()   
        print("---------------Event queue-------------------------")
        for eve in self.event_pq:
            eve.display()
        self.s.display()
        print("---------------Job queue-------------------------")
        self.s.job_que.display()
        print("===================================================")
        

In [51]:
class Users:
    def __init__(self, user_id, sim):
#         print("User intialising...")
        self.id = user_id
        self.state = THINK
        self.req_dropped = 0
        self.req_timedout = 0
        self.waitt_list = []
        self.t_req = 0
        self.tp_timeout = 0
        self.tp_service = 0
        self.generate_request(sim)
        
    def display(self):
        print("User",self.id,":-", "State:",self.state, "Request time :",self.t_req, "Service time :",self.tp_service)  

    def generate_thinktime(self):
        return random.triangular(3,6,4)
#         return np.random.exponential(3)
    
    def generate_service(self):
        return random.uniform(3,9)
#         return np.random.exponential(6)
    
    def generate_timeout(self):
        return (10)
    
    def generate_request(self, sim):
#         print("generate_request()")
        self.t_req = sim.clock + self.generate_thinktime()
        self.tp_timeout = self.generate_timeout()
        self.tp_service = self.generate_service()
        
        sim.req_num += 1
        self.req = Request(sim.req_num, self.id, self.t_req, self.tp_timeout, self.tp_service)
        event = Event(ARR, self.req, self.t_req)
        
        heapq.heappush(sim.event_pq, event)

    def put_in_queue(self, sim):
#         print("put_in_queue(): user: ",self.id,"user of req:",self.req.user)
        if(not(sim.que.isFull())):
            sim.que.enqueue(self.req)
            self.req.start_wait_t = sim.clock
            self.state = REQ
            self.t_req = float('inf')
            return True
        else:
            self.req_dropped += 1
            self.generate_request(sim)
            return False

    def request_finish(self,tstamp):
#         print("request_finish()")
        self.waitt_list.append(self.req.tp_wait)
        self.state = THINK
        self.generate_request(sim)


In [52]:
class Event:
    def __init__(self, e_type, req, timestamp):
#         print("Event intialising...:","type",e_type,"user", req.user,"stmp",timestamp)
        self.e_type = e_type
        self.req = req
        self.timestamp = timestamp


    def __lt__(self, other):
        return self.timestamp < other.timestamp
    
    def display(self):
        print("Event type",self.e_type,"Timestamp:",self.timestamp,"Req of user:",self.req.user)


In [53]:
class Core:
    def __init__(self, core_id ,sw_time):
#         print("core instialsiing...:",core_id)
#         self.req = -1
        self.core_id = core_id  
        self.state = IDLE
        self.switch_time = sw_time
    def display(self):
        print("Core:",self.core_id,"Core state:",self.state)
        
    def as_request(self, req):
#         print("as_request() of user:",req.user)
        self.req = req
        self.state = BUSY

    def n_event(self, tstmp):
#         print("n_event(), core_id:",self.core_id,"clock:",tstmp,"service:",self.req.tp_service)
        if self.switch_time < self.req.tp_service:
            return Event(SWCH, self.req, tstmp + self.switch_time)
        else:
            return Event(DEP, self.req, tstmp + self.req.tp_service)

In [54]:
class Server:
    def __init__(self,cores):
        print("Server Instialisong...")
        self.state = IDLE
        self.no_of_cores = cores
        self.cores_list = [Core(i,2) for i in range(self.no_of_cores)]
        self.job_que = Queue()
        self.n_reqs = 0
        self.max_reqs = 10
    
    def display(self):
        print("No of request in server:", self.n_reqs)
        print("No of cores:", len(self.cores_list))
        print("-----------Cores--------------------")
        for c in self.cores_list:
            c.display()
        
    def serve(self, sim, event):
        if event.e_type == DEP:
            print("------Serving DEP event------")
            sim.user[event.req.user].request_finish(sim.clock)
            self.n_reqs -= 1
            core = self.cores_list[event.req.core_num]
            if not(sim.que.isEmpty()):
                req = sim.que.dequeue()
                self.n_reqs += 1
                self.job_que.enqueue(req)
                
            if self.job_que.isEmpty():
                core.state = IDLE
                print("Server idle.")
                # sim.n_depart = float('inf')
                return
            
            ass_req = self.job_que.dequeue()
            core.as_request(ass_req)
            ass_req.assign_core(core.core_id)

            n_event = core.n_event(sim.clock)
            heapq.heappush(sim.event_pq, n_event)
            return

        elif event.e_type == ARR:
            print("---Serving ARR event----")
#             print("n_reqs:",self.n_reqs,"max_reqs:",self.max_reqs)
            if self.n_reqs < self.max_reqs:
                req = sim.que.dequeue()
                self.n_reqs += 1
                req.add_to_wait(sim.clock)
                self.job_que.enqueue(req)
                if self.n_reqs <= self.no_of_cores:
                    for c in self.cores_list:
                        if c.state == IDLE:
                            print("Core found idle.")
                            ass_req = self.job_que.dequeue()
                            c.as_request(ass_req)
                            ass_req.assign_core(c.core_id)

                            n_event = c.n_event(sim.clock)
                            heapq.heappush(sim.event_pq, n_event)
                            break
                else:
                    print("All cores busy.")
            return

        elif event.e_type == SWCH:
            print("---Serving SWITCH event----")
            core = self.cores_list[event.req.core_num]
            event.req.dec_time(core.switch_time)
            self.job_que.enqueue(event.req)

            ass_req = self.job_que.dequeue()
            core.as_request(ass_req)
            ass_req.assign_core(core.core_id)

            n_event = core.n_event(sim.clock)
            heapq.heappush(sim.event_pq, n_event)
            return
